<a href="https://colab.research.google.com/github/MYadav111/HateSpeech/blob/main/SBMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Data and Essential Libraries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from nltk.util import pr
import re
import nltk
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
stopword=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from matplotlib import style

In [ ]:
tweet = pd.read_csv("/content/labeled_data original.csv")
print(tweet.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/labeled_data original.csv'

In [ ]:
tweet.info()

In [ ]:
#Mapping the Data
tweet["labels"] = tweet["class"].map({0: "Hate Speech",
                                    1: "Offensive Language",
                                    2: "No Hate and Offensive"})
print(tweet.head())

In [ ]:
tweet['labels'].value_counts()

In [ ]:
tweet.isnull().sum()

In [ ]:
#Removing Unwanted Columns
tweet = tweet[["tweet", "labels"]]
print(tweet.head())

#Data Cleaning

In [ ]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
tweet["tweet"] = tweet["tweet"].apply(clean)

In [ ]:
tweet.head()

#Model Training

In [ ]:
#Decision Tree Classifier
x = np.array(tweet["tweet"])
y = np.array(tweet["labels"])

cv_DT = CountVectorizer()

X = cv_DT.fit_transform(x) # Fit the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

clf_DT = DecisionTreeClassifier()
clf_DT.fit(X_train,y_train)

In [ ]:
y_pred_DT = clf_DT.predict(X_test)

cm = confusion_matrix(y_test, y_pred_DT)
print(cm)

acc_DT = accuracy_score(y_test, y_pred_DT)
print("\n")
print("Accuracy of Decision Tree Classifier is: ", acc_DT)

In [ ]:
#Prediction using Decision Tree Classifier
sample = "Let's unite and kill all the people who are protesting against the government"
data = cv_DT.transform([sample]).toarray()
print(clf_DT.predict(data))

In [ ]:
#Support Vector Classifier
cv_SVC = CountVectorizer()

X = cv_SVC.fit_transform(x) # Fit the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y, shuffle=True)

clf_SVC = SVC()

clf_SVC = SVC(kernel = 'linear', random_state = 0)
clf_SVC.fit(X_train,y_train)

In [ ]:
y_pred_SVC = clf_SVC.predict(X_test)

cm = confusion_matrix(y_test, y_pred_SVC)
print(cm)

acc_SVC = accuracy_score(y_test, y_pred_SVC)
print("\n")
print("Accuracy of Support Vector Classifier is: ", acc_SVC)

In [ ]:
#Prediction using Support Vector Classifier
sample1 = "Happy Everyone"
data1 = cv_SVC.transform([sample]).toarray()
print(clf_SVC.predict(data1))

In [ ]:
# Naive Bayes
cv_NB = CountVectorizer()

X = cv_NB.fit_transform(x) # Fit the Data
X = X.toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y, shuffle=True)

clf_NB = GaussianNB()
clf_NB.fit(X_train, y_train)

In [ ]:
y_pred_NB = clf_NB.predict(X_test)

cm = confusion_matrix(y_test, y_pred_NB)
print(cm)

acc_NB = accuracy_score(y_test, y_pred_NB)
print("\n")
print("Accuracy of Naive Bayes is: ", acc_NB)

In [ ]:
# Random Forest Classifier
cv_RF = CountVectorizer()

X = cv_RF.fit_transform(x) # Fit the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y, shuffle=True)

clf_RF = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
clf_RF.fit(X_train, y_train)

In [ ]:
y_pred_RF = clf_RF.predict(X_test)

cm = confusion_matrix(y_test, y_pred_RF)
print(cm)

acc_RF = accuracy_score(y_test, y_pred_RF)
print("\n")
print("Accuracy of Random Forest Classifier is: ", acc_RF)

#Accuracy of Different ML Model

In [ ]:
print("Accuracy of Decision Tree Classifier is: ", acc_DT)
print("\n")
print("Accuracy of Random Tree Classifier is: ", acc_RF)
print("\n")
print("Accuracy of Support Vector Classifier is: ", acc_SVC)
print("\n")
print("Accuracy of Naive Bayes is: ", acc_NB)

#Data Visualization

In [ ]:
style.use('ggplot')

In [ ]:
fig = plt.figure(figsize=(10,5))
sns.countplot(x='labels', data = tweet)

In [ ]:
fig = plt.figure(figsize=(7,7))
colors = ("red", "gold", "white")
wp = {'linewidth':2, 'edgecolor':"black"}
tags = tweet['labels'].value_counts()
explode = (0.1, 0.1, 0.1)
tags.plot(kind='pie',autopct = '%1.1f%%', shadow=True, colors = colors, startangle =90,
         wedgeprops = wp, explode = explode, label='')
plt.title('Distribution of sentiments')

In [ ]:
non_hate_tweets = tweet[tweet.labels == "No Hate and Offensive"]
non_hate_tweets.head()

In [ ]:
text = ' '.join([word for word in non_hate_tweets['tweet']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Most frequent words in non hate tweets', fontsize = 19)
plt.show()

In [ ]:
hate_tweets = tweet[tweet.labels == "Hate Speech"]
hate_tweets.head()

In [ ]:
text = ' '.join([word for word in hate_tweets['tweet']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Most frequent words in hate tweets', fontsize = 19)
plt.show()

In [ ]:
offensive_tweets = tweet[tweet.labels == "Offensive Language"]
offensive_tweets.head()

In [ ]:
text = ' '.join([word for word in offensive_tweets['tweet']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Most frequent words in Offensive Language', fontsize = 19)
plt.show()

#Exporting Model

In [ ]:
import pickle

In [ ]:
filename = 'testsvc.sav'
pickle.dump(clf_SVC, open(filename, 'wb'))

In [ ]:
import numpy as np

# Assuming x is a NumPy array
x = np.array([[1,2]])

column_names = ['id', 'label', 'tweet']
for column_name, column in zip(column_names, x.T):
    print(column_name, ":", column)